In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from config import api_key

In [2]:
from census import Census
from us import states

In [3]:
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

* As only civilians are legel to vote, it would make sense to only pull the civilian employment data.
* The labor force is the sum of employed and unemployed persons
* Columns data to pull: 
* Income variables: 
    "income": B19013_001E, "income_per_capita": B19301_001E
* Employment variables:
    "employable civilians": B23025_003E, "employed civilians": B23025_004E, "unemployed civilians": B23025_005E
* Population variables: 
    "Total population": B01003_001E
* Poverty variables:
    "poverty people": "B17001_002E"
    #"poverty_family": "B17012_002E"
    
* Age variables:
    "median age": "B01002_001E" 

In [4]:
# Census API calls
c = Census(api_key, year=2018)
raw_data = c.acs5.get(('NAME', 'B19013_001E', 'B19301_001E', 'B23025_003E',
                       'B23025_004E', 'B23025_005E', 'B01003_001E', "B01002_001E", "B17001_002E"), {'for':'state:*'})
census_df_raw = pd.DataFrame(raw_data)
census_df_raw = census_df_raw.rename(columns = {'state': 'State#',
                          'NAME': 'State', 
                          'B01003_001E':'Total_population',
                          'B23025_003E':'Employable_civilians',
                          'B23025_004E':'Employed_civilians', 
                          'B23025_005E':'Unemployed_civilians', 
                          'B19013_001E':'Income_median',
                          'B19301_001E':'Income_per_capita',
                          "B01002_001E":'median age',
                          "B17001_002E":'Poverty_count'    
})
census_df_raw = census_df_raw.sort_values('State#').reset_index(drop=True)
# census_df_raw = census_df_raw[['State', 'Total_population','Income_per_capita', 'Employable_civilians', 'Unemployed_civilians','Poverty_count'  ]]
census_df_raw.head()

,State,Income_median,Income_per_capita,Employable_civilians,Employed_civilians,Unemployed_civilians,Total_population,median age,Poverty_count,State#
0,Alabama,48486.0,26846.0,2224606.0,2076708.0,147898.0,4864680.0,38.9,829400.0,01
1,Alaska,76715.0,35874.0,379219.0,351152.0,28067.0,738516.0,34.0,77865.0,02
2,Arizona,56213.0,29265.0,3256192.0,3045978.0,210214.0,6946685.0,37.4,1092192.0,04
3,Arkansas,45726.0,25635.0,1368037.0,1292263.0,75774.0,2990671.0,37.9,510337.0,05
4,California,71228.0,35021.0,19630514.0,18309012.0,1321502.0,39148760.0,36.3,5487141.0,06


In [5]:
# Making a summary table 
census_df = census_df_raw.loc[:,['State', 'Total_population', 'Income_median', 'Income_per_capita']]
census_df['Poverty_rate'] = census_df_raw.Poverty_count/census_df_raw.Total_population
census_df['Unemployment_rate'] = census_df_raw.Unemployed_civilians/census_df_raw.Employable_civilians
state_df = pd.read_csv('Resources/state_centroids.csv')
census_df = pd.merge(census_df, state_df, on = 'State')
census_df = census_df.set_index('State')
census_df.head()

,Total_population,Income_median,Income_per_capita,Poverty_rate,Unemployment_rate,Latitude,Longitude
State,,,,,,,
Alabama,4864680.0,48486.0,26846.0,0.170494,0.066483,32.7794,-86.8287
Alaska,738516.0,76715.0,35874.0,0.105434,0.074013,64.0685,-152.2782
Arizona,6946685.0,56213.0,29265.0,0.157225,0.064558,34.2744,-111.6602
Arkansas,2990671.0,45726.0,25635.0,0.170643,0.055389,34.8938,-92.4426
California,39148760.0,71228.0,35021.0,0.140161,0.067319,37.1841,-119.4696


In [ ]:
# Visualization via bar plots: population, median income, per capita income, poverty rate, unemployment rate

plt.subplot(511)
fig_population = census_df['Total_population'].sort_values().plot(kind='bar',figsize=(15,20), legend=True)

plt.subplot(512)
fig_income = census_df['Income_median'].sort_values().plot(kind='bar', figsize=(15,20), legend=True)

plt.subplot(513)
fig_capita_income = census_df['Income_per_capita'].sort_values().plot(kind='bar',figsize=(15,20), legend=True)

plt.subplot(514)
fig_unemploy_rate = census_df['Unemployment_rate'].sort_values().plot(kind='bar',figsize=(15,20), legend=True)

plt.subplot(515)
fig_unemploy_rate = census_df['Poverty_rate'].sort_values().plot(kind='bar',figsize=(15,20), legend=True)

plt.tight_layout()

In [6]:
import gmaps
from config import gkey
gmaps.configure(api_key=gkey)

In [23]:
# Settings for gmap figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}

# locations of all states
state_coords = census_df.loc[:, ['Latitude', 'Longitude']]

In [27]:
# Population heatmap
# markers = gmaps.marker_layer(state_coords)
# gmap_fig.add_layer(markers)

gmap_fig = gmaps.figure(figure_layout,map_type='HYBRID')
population_layer = gmaps.heatmap_layer(state_coords, weights = census_df['Total_population'],max_intensity = census_df['Total_population'].max(),
                                  dissipating=False, point_radius=3, opacity=0.5)
gmap_fig.add_layer(population_layer)
gmap_fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# median income heatmap
gmap_fig = gmaps.figure(figure_layout,map_type='HYBRID')
income_layer = gmaps.heatmap_layer(state_coords, weights = census_df['Income_median'],max_intensity = census_df['Income_median'].max(),
                                  dissipating=False, point_radius=3, opacity=0.3)

gmap_fig.add_layer(income_layer)
gmap_fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# per_capita_income heatmap
gmap_fig = gmaps.figure(figure_layout,map_type='HYBRID')
per_capita_income_layer = gmaps.heatmap_layer(state_coords, weights = census_df['Income_per_capita'],max_intensity = census_df['Income_per_capita'].max(),
                                  dissipating=False, point_radius=3, opacity=0.3)
gmap_fig.add_layer(per_capita_income_layer)
gmap_fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
# Unemployment_rate heatmap
gmap_fig = gmaps.figure(figure_layout,map_type='HYBRID')
unemploy_layer = gmaps.heatmap_layer(state_coords, weights = census_df['Unemployment_rate'], max_intensity = census_df['Unemployment_rate'].max(),
                                  dissipating=False, point_radius=4, opacity=0.6)

gmap_fig.add_layer(unemploy_layer)
gmap_fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# poverty rate heatmap
gmap_fig = gmaps.figure(figure_layout,map_type='HYBRID')
poverty_layer = gmaps.heatmap_layer(state_coords, weights = census_df['Poverty_rate'],max_intensity = census_df['Poverty_rate'].max(),
                                  dissipating=False, point_radius=4, opacity=0.6)

gmap_fig.add_layer(poverty_layer)
gmap_fig

Figure(layout=FigureLayout(height='420px'))